# Build a wine quality prediction model using K-NN
## Determine efficacy of scaling

In [1]:
# Imports....the usual suspects
import postgresql
import pandas as pd
import sklearn
import matplotlib
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error

## 1. Download the Wine Quality Dataset from the UCI
http://archive.ics.uci.edu/ml/datasets/Wine+Quality
Look for the 'Data Folder' link near top

In [2]:
# Grab the data, put in DataFrame
# This is the US Census data on incomes
df = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', delimiter=';')
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [3]:
df.to_pickle('df.pkl')

In [4]:
del(df)
df

NameError: name 'df' is not defined

In [5]:
df = pd.read_pickle('df.pkl')
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


## 2. Split the Red Wine Dataset into training and testing

In [6]:
#Dropping nas
df = df.dropna()
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [7]:
#Changing the datatypes
for column in df.columns:
 if df[column].dtype == type(object):
  le = sklearn.preprocessing.LabelEncoder()
  df[column] = le.fit_transform(df[column])

In [8]:
x_df = df.drop(columns=['quality'])
y_df = df['quality']
x_train, x_test, y_train, y_test = train_test_split(x_df, y_df)
df.tail()
x_train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
239,8.2,1.000,0.09,2.3,0.065,7.0,37.0,0.99685,3.32,0.55,9.0
523,9.3,0.400,0.49,2.5,0.085,38.0,142.0,0.99780,3.22,0.55,9.4
943,9.8,0.500,0.34,2.3,0.094,10.0,45.0,0.99864,3.24,0.60,9.7
1293,7.5,0.755,0.00,1.9,0.084,6.0,12.0,0.99672,3.34,0.49,9.7
188,7.9,0.500,0.33,2.0,0.084,15.0,143.0,0.99680,3.20,0.55,9.5


## 3. Build a K-NN Regression Model to predict the continuous quality variable in the Red Wine Dataset.  Predict for your test set (you're estimating quality column)

Hint, you may want to drop the last element of your test set if it's a giant array: red_y_test[:-1]

In [9]:
#5 Neighbors Model
clf = KNeighborsRegressor(n_neighbors=5, p=1)
clf.fit(x_train, y_train)
print('Accuracy: {:.2f}'.format(clf.score(x_test, y_test)))

Accuracy: 0.17


In [10]:
#10 Neighbors Model
clf = KNeighborsRegressor(n_neighbors=10, p=1)
clf.fit(x_train, y_train)
print('Accuracy: {:.2f}'.format(clf.score(x_test, y_test)))

Accuracy: 0.16


## 4. What is the mean of the differences between the Y actuals (quality column) and your rounded predictions?

In [11]:
#Method 2
y_pred = pd.DataFrame(clf.predict(x_test), columns = ['prediction'])
y_pred['actual'] = y_test.values
y_pred['diff'] = y_pred['actual'] - y_pred['prediction']
y_pred['diff'].abs().mean()

0.5739999999999996

## 5. Repeat steps 2-5 after scaling the data.  Try at least two types of scaling.  

# Method 1: Standard Scaler

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_df_scaled = scaler.fit_transform(x_df)
x_train, x_test, y_train, y_test = train_test_split(x_df_scaled, y_df)

In [13]:
#2. Split the Red Wine Dataset into training and testing
#Dropping nas
df = df.dropna()
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [14]:
#3. Build a K-NN Regression Model to predict the continuous quality variable in 
#the Red Wine Dataset. Predict for your test set (you're estimating quality column
#5 Neighbors Model
clf = KNeighborsRegressor(n_neighbors=5, p=1)
clf.fit(x_train, y_train)
print('Accuracy: {:.2f}'.format(clf.score(x_test, y_test)))

Accuracy: 0.26


In [15]:
#10 Neighbors Model
clf = KNeighborsRegressor(n_neighbors=10, p=1)
clf.fit(x_train, y_train)
print('Accuracy: {:.2f}'.format(clf.score(x_test, y_test)))

Accuracy: 0.32


In [16]:
#4. What is the mean of the differences between the Y actuals 
#(quality column) and your rounded predictions?
y_pred = pd.DataFrame(clf.predict(x_test), columns = ['prediction'])
y_pred['actual'] = y_test.values
y_pred['diff'] = y_pred['actual'] - y_pred['prediction']
y_pred['diff'].abs().mean()

0.5077499999999997

# Method 2: Robust Scaler

In [17]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
x_df_scaled = scaler.fit_transform(x_df)
x_train, x_test, y_train, y_test = train_test_split(x_df_scaled, y_df)

In [18]:
#2. Split the Red Wine Dataset into training and testing
#Dropping nas
df = df.dropna()
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [19]:
#3. Build a K-NN Regression Model to predict the continuous quality variable in 
#the Red Wine Dataset. Predict for your test set (you're estimating quality column
#5 Neighbors Model
clf = KNeighborsRegressor(n_neighbors=5, p=1)
clf.fit(x_train, y_train)
print('Accuracy: {:.2f}'.format(clf.score(x_test, y_test)))

Accuracy: 0.33


In [20]:
#10 Neighbors Model
clf = KNeighborsRegressor(n_neighbors=10, p=1)
clf.fit(x_train, y_train)
print('Accuracy: {:.2f}'.format(clf.score(x_test, y_test)))

Accuracy: 0.32


In [21]:
#4. What is the mean of the differences between the Y actuals 
#(quality column) and your rounded predictions?
y_pred = pd.DataFrame(clf.predict(x_test), columns = ['prediction'])
y_pred['actual'] = y_test.values
y_pred['diff'] = y_pred['actual'] - y_pred['prediction']
y_pred['diff'].abs().mean()

0.5049999999999998

## 6. Did Scaling help?

In [22]:
'Without scaling the accuracy is about 0.25. With Standard Scaler it is about 0.30. With Robust Scaler it is about 0.32. We can see that Scaling definitely helps the accuracy of the prediction. It makes a significant difference.'

'Without scaling the accuracy is about 0.25. With Standard Scaler it is about 0.30. With Robust Scaler it is about 0.32. We can see that Scaling definitely helps the accuracy of the prediction. It makes a significant difference.'